In [1]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from transformers import RobertaTokenizer


In [2]:
train = pd.read_csv('../input/tweet-sentiment-extraction/train_folds.csv')
test = pd.read_csv('../input/tweet-sentiment-extraction/test.csv')

In [3]:
origin = pd.read_csv('../input/tweet_dataset.csv')

In [4]:
train.head()

,textID,text,selected_text,sentiment,kfold
0,f7fdea625a,i`m so bored i can barely even tweet. i have n...,bored,negative,0
1,c19de2c75b,awwwwwwwwww thats jus...awwwww.did she get t...,awwwwwwwwww thats jus...awwwww.did she get to...,neutral,0
2,0f963af18f,I did not twitt yesterday cause it was a very ...,I can not sleep,negative,0
3,0583c78cc1,Congratulation`s to phil packer on completing ...,Congratulation`s,positive,0
4,1cdb444ea5,O`Charleys? Pretty good. Especially when its f...,Pretty good.,positive,0


In [5]:
origin.head()

,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text
0,1956967341,empty,xoshayzers,i know i was listenin to bad habit earlier a...,@tiffanylue i know i was listenin to bad habi...,p1000000000,NaN,NaN
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin o...,c811396dc2,negative,headache
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...,9063631ab1,negative,gloomy
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!,2a815f151d,positive,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,We want to trade with someone who has Houston...,@dannycastillo We want to trade with someone w...,82565a56d3,neutral,We want to trade with someone who has Houston ...


In [15]:
origin['author'].nunique()

33871

In [16]:
origin.shape

(40000, 8)

In [18]:
origin.groupby('author').size().sort_values()

author
000yea000           1
joshuak92           1
joshuajcohen        1
joshuabruce         1
joshuaarnao         1
joshsandoval        1
joshpatil           1
joshnorgren         1
joshlim             1
joshjones_75        1
joshigginbotham     1
joshie1226          1
joshgard            1
joshg85             1
joshftv             1
joshfittell         1
joshdebauche        1
joshdavid           1
joshcoop            1
joshboulton         1
josh_kim            1
josh3333            1
josephrueter        1
josephroque         1
josephranseth       1
josephmoney         1
josephlavigne       1
josephcoster        1
joshuamange         1
josephchristian     1
                   ..
Gemmaboyle         10
keza34             10
sweetdreamer       10
HelloLizzi         10
HanaStephenson     10
Samm_xo            10
Jinxie_G           10
violetbakes        10
beingnobody        11
MandyPandy32       11
Tittch             11
ComedyQueen        11
TraceyHewins       12
lesley007          12
lop

In [19]:
origin[origin['author']=='MissxMarisa']

,textID,old_sentiment,author,text,old_text,aux_id,new_sentiment,selected_text
260,1957023061,surprise,MissxMarisa,"Im sorry I don`t think I stick out, at least...",@freshypanda Im sorry I don't think I stick o...,dbf11e3d70,neutral,"Im sorry I don`t think I stick out, at least ..."
333,1957041812,hate,MissxMarisa,"Hahahaha! It`s not horrible, if others were s...","@meganmansyn Hahahaha! It's not horrible, if o...",3949709cf1,positive,"not horrible,"
1941,1957434995,neutral,MissxMarisa,This year,@tsarnick This year,a68443a78b,neutral,This year
2143,1957479923,worry,MissxMarisa,I`m concerned for that family,@Zobi09 I'm concerned for that family,2524332d66,positive,NaN
2298,1957514298,neutral,MissxMarisa,"Yeah, that was my thumb last night. Sorry","@tsarnick Yeah, that was my thumb last night. ...",93338c8415,negative,Sorry
21267,1694202109,love,MissxMarisa,Thank you I liked Joe`s clothes` being rippe...,@shortyyyy Thank you I liked Joe's clothes' b...,e858f93fc8,positive,I liked
21560,1694256899,surprise,MissxMarisa,Bill can take me to Fangtasia and hit this sh...,@Zanna85 Bill can take me to Fangtasia and hit...,p1000004778,NaN,NaN
22574,1694455792,happiness,MissxMarisa,The cast of True Blood was there,@Zobi09 The cast of True Blood was there,197bec62c7,neutral,NaN
22773,1694493269,sadness,MissxMarisa,"Haha yes I do it`s hard work, too! I didn`t ...","@Zobi09 Haha yes I do it's hard work, too! I ...",5c3ac415c3,neutral,"Haha yes I do it`s hard work, too! I didn`t g..."
23154,1694568031,neutral,MissxMarisa,California,@TheLonely California,84c82f15ba,neutral,California


In [6]:
origin.rename(index=str, columns={'sentiment':'old_sentiment'}, inplace=True)

In [7]:
train = train.merge(origin[['old_sentiment','aux_id']], left_on='textID', right_on='aux_id', how='left')

In [8]:
train.head()

,textID,text,selected_text,sentiment,kfold,old_sentiment,aux_id
0,f7fdea625a,i`m so bored i can barely even tweet. i have n...,bored,negative,0,sadness,f7fdea625a
1,c19de2c75b,awwwwwwwwww thats jus...awwwww.did she get t...,awwwwwwwwww thats jus...awwwww.did she get to...,neutral,0,surprise,c19de2c75b
2,0f963af18f,I did not twitt yesterday cause it was a very ...,I can not sleep,negative,0,worry,0f963af18f
3,0583c78cc1,Congratulation`s to phil packer on completing ...,Congratulation`s,positive,0,enthusiasm,0583c78cc1
4,1cdb444ea5,O`Charleys? Pretty good. Especially when its f...,Pretty good.,positive,0,love,1cdb444ea5


In [9]:
sum(train['old_sentiment'].isnull())

0

In [10]:
test = test.merge(origin[['old_sentiment','aux_id']], left_on='textID', right_on='aux_id', how='left')

In [11]:
sum(test['old_sentiment'].isnull())

0

In [12]:
test.head()

,textID,text,sentiment,old_sentiment,aux_id
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,fun,f87dea47db
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,relief,96d74cb729
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,sadness,eee518ae67
3,01082688c6,happy bday!,positive,enthusiasm,01082688c6
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,happiness,33987a8ee5


In [13]:
train.to_csv('../input/tweet-sentiment-extraction/train_folds_with_o.csv')